In [1]:
!pip install transformers datasets PyPDF2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.3 MB/s eta 0:00:00


In [2]:
from google.colab import files
uploaded = files.upload()

Saving sophies.pdf to sophies.pdf


In [3]:
import PyPDF2

def extract_text_from_pdf(path):
    with open(path, 'rb') as f:
        reader = PyPDF2.PdfReader(f)
        text = ''
        for page in reader.pages:
            text += page.extract_text()
        return text

pdf_text = extract_text_from_pdf("sophies.pdf")


In [7]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token



In [5]:
# /root/.cache/huggingface/transformers/
!ls /root/.cache/huggingface/transformers/


ls: cannot access '/root/.cache/huggingface/transformers/': No such file or directory


In [8]:
from torch.utils.data import Dataset, DataLoader
import torch

class TextDataset(Dataset):
    def __init__(self, text, tokenizer, block_size=128):
        tokens = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
        input_ids = tokens['input_ids'][0]
        self.samples = [input_ids[i:i+block_size] for i in range(0, len(input_ids)-block_size, block_size)]

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        x = self.samples[idx]
        return {'input_ids': x, 'labels': x}

dataset = TextDataset(pdf_text, tokenizer)
loader = DataLoader(dataset, batch_size=1, shuffle=True)


In [10]:
# from transformers import AdamW
from torch.optim import AdamW # Import AdamW from torch.optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.train()

optimizer = AdamW(model.parameters(), lr=5e-5)

for epoch in range(1):
    for batch in loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        print(f"Loss: {loss.item():.4f}")


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Loss: 5.5262
Loss: 4.8539
Loss: 4.6579
Loss: 5.2749
Loss: 4.3462
Loss: 4.3736
Loss: 4.6408


In [11]:
model.eval()
prompt = "Once upon a time"
inputs = tokenizer(prompt, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_length=200, do_sample=True, top_k=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Once upon a time in the history of human beings, most of us were aware of a world in which people were working in the mines, collecting the iron ore, building and repairing buildings. In that world, we were constantly surrounded by our own prejudices and prejudices that were not in harmony with our own. The most natural way to live and to learn was to work in the mines. But in that respect, it was to be a slave—a slave to the mines'.

So as we learned to care for ourselves, we moved around these mines or dug under them at intervals, making sure that there was, after all, something left to look forward the next day.

There is no evidence that we were "born" (read: not by working in the mines until puberty, or maybe earlier) in the process. Our "own" nature didn't explain its existence. The first sign of our descent into servitude—the human kind—"is that we didn't know it
